This notebook contains code to use GPT4 to generate test cases for Python programs extracted from the code-alpaca dataset.

[Code alpaca 20k dataset](https://github.com/sahil280114/codealpaca/tree/master/data)

Contains a set of 20 contains 20K instruction-following data used for fine-tuning the Code Alpaca model. This JSON file is a list of dictionaries, each dictionary contains the following fields:

- `instruction`: str, describes the task the model should perform. Each of the 20K instructions is unique.
- `input`: str, optional context or input for the task. For example, when the instruction is "Amend the following SQL query to select distinct elements", the input is the SQL query. Around 40% of the examples have an input.
- `output`: str, the answer to the instruction as generated by text-davinci-003.


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
#!wget https://github.com/sahil280114/codealpaca/raw/master/data/code_alpaca_20k.json -O drive/MyDrive/tmp/code_alpaca_20k.json

--2023-09-23 04:02:17--  https://github.com/sahil280114/codealpaca/raw/master/data/code_alpaca_20k.json
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sahil280114/codealpaca/master/data/code_alpaca_20k.json [following]
--2023-09-23 04:02:18--  https://raw.githubusercontent.com/sahil280114/codealpaca/master/data/code_alpaca_20k.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8057896 (7.7M) [text/plain]
Saving to: ‘drive/MyDrive/tmp/code_alpaca_20k.json’

drive/MyDrive/tmp/c 100%[===================>]   7.68M  --.-KB/s    in 0.1s    

2023-09-23 04:02:18 (68.5 MB/s) - ‘drive/MyDrive/tmp/code

In [ ]:
import json
with open('drive/MyDrive/tmp/code_alpaca_20k.json') as fin:
  alpaca = json.load(fin)

In [ ]:
# extract the data where output is Python program
alpaca_python = [x for x in alpaca if x['output'].startswith('def')]

In [ ]:
len(alpaca_python)

4137

In [ ]:
import openai

openai.api_type = "azure"
openai.api_base = 'https://YOUR_API_BASE.openai.azure.com/'
openai.api_version = "2023-07-01-preview"
openai.api_key = 'YOUR_API_KEY'


In [ ]:
def make_prompt(problem):
  prompt = f"""
  Instruction:
  {problem['instruction']}

  Solution:
  {problem['output']}

  Example input:
  {problem['input']}

  Given the above instruction, solution, and example input to a programming problem, generate test cases for the program.
  Write each test case as a seperate Python function with meaningful name. The output should be enclosed in triple ticks (```).

  Example output:
  ```
  def test_not_null():
    # code goes here
    assert True

  def test_correct():
    # code goes here
    assert a = b
  ```
  """
  return prompt


In [ ]:
def gen_test_cases(prompt):
  """Call OpenAI API and extract test cases."""
  try:
    response = openai.ChatCompletion.create(
        deployment_id='gpt40-20230827',
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.2, max_tokens=500, top_p=0.95
    )
    return response['choices'][0]['message']['content']
  except Exception as ex:
    print(ex)
    return None


In [ ]:
def extract_functions(response) -> str:
  """Extract functions from the response."""
  try:
    # take response starting from the first def (to ignore case where the responses starts with ```python)
    def_pos = response.index('def ')
    if def_pos:
      response = response[def_pos:]
    response = response.replace('```', '')
    return response
  except Exception as ex:
    print(ex)
    return None

In [ ]:
import os

output_file = 'drive/MyDrive/tmp/code_alpaca_with_test_cases.json'
if os.path.exists(output_file):
  with open(output_file, encoding='utf-8') as fin:
    dataset = json.load(fin)
else:
  dataset = []


In [ ]:
len(dataset)

1681

In [ ]:
import os

output_file = 'drive/MyDrive/tmp/code_alpaca_with_test_cases.json'
if os.path.exists(output_file):
  with open(output_file, encoding='utf-8') as fin:
    dataset = json.load(fin)
else:
  dataset = []

for i, problem in enumerate(alpaca_python[len(dataset):]):
  prompt = make_prompt(problem)
  print(i, problem['instruction'])
  test_cases = gen_test_cases(prompt)
  if test_cases:
    test_cases = extract_functions(test_cases)
  problem['test_cases'] = test_cases
  dataset.append(problem)
  # save every 20 problems in case of failure
  if i % 20 == 0:
    with open(output_file, encoding='utf-8', mode='w') as fout:
      json.dump(dataset, fout)


0 Create a function that takes two strings, s1 and s2, and checks to see if s2 is a substring of s1
1 For a given list of numbers, write a palindrome checker.
2 Create a function that takes an array of string elements and sorts them in alphabetical order.
3 Write a Python code to search for a given keyword in a dictionary.
4 Write a python to convert an IP address from dot-decimal notation to binary notation.
5 Write a code to generate a list with the first 20 Fibonacci numbers.
6 Create a function to return all possible sums of two numbers.
7 Rewrite the code to be efficient and use less lines of code.
8 Implement quicksort algorithm in Python for a list of integers.
9 Generate the Fibonacci sequence using a for loop in Python.
10 Write a python program that takes in two input strings and checks if the second string is an anagram of the first one.
11 Using the comments as a guide, fill in the blanks with code where applicable to make the function jumbledLetters() work.
12 Write a Ruby

In [ ]:
with open('drive/MyDrive/tmp/code_alpaca_with_test_cases.json', encoding='utf-8', mode='w') as fout:
  json.dump(dataset, fout)

In [ ]:
dataset

[{'instruction': 'Write a replace method for a string class which replaces the given string with a given set of characters.',
  'input': 'string = "Hello World!"\nreplace_with = "Greetings!"',
  'output': 'def replace(self, replace_with):\n    new_string = ""\n    for char in self:\n        if char == " ":\n            new_string += replace_with\n        else:\n            new_string += char\n    return new_string',
  'test_cases': 'def test_replace_with_empty_string():\n    string = "Hello World!"\n    replace_with = ""\n    assert string.replace(replace_with) == "HelloWorld!"\n\ndef test_replace_with_same_string():\n    string = "Hello World!"\n    replace_with = "Hello World!"\n    assert string.replace(replace_with) == "Hello World!Hello World!"\n\ndef test_replace_with_special_characters():\n    string = "Hello World!"\n    replace_with = "@#$%^&*()"\n    assert string.replace(replace_with) == "Hello@#$%^&*()World!"\n\ndef test_replace_with_numbers():\n    string = "Hello World!"\